## Data Cleaning Summary

This notebook contains the cleaning process of four healthcare-related datasets.  
The raw data was messy, inconsistent, and contained null values, incorrect formatting, and redundancy.

### Cleaned Tables:
1. `patients_clean`
2. `treatments_clean` (merged with adverse reactions)
3. `treatments_cut_clean`
4. `adverse_reactions_clean` (used for merging only)

The goal of this cleaning process was to prepare the data for reliable analysis and dashboard creation 

# Importing Required Libraries


In [9]:
import pandas as pd 
import numpy as np 
import re

# Loading the Datasets

In [12]:
patients = pd.read_csv('patients.csv')
treatments = pd.read_csv('treatments.csv')
treatments_cut = pd.read_csv('treatments_cut.csv')
adverse_reactions = pd.read_csv('adverse_reactions.csv')

# Previewing the Dataset

In [1441]:
patients.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [1443]:
treatments.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [1446]:
treatments_cut.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,NaN
2,alwin,svensson,36u - 39u,-,7.78,7.34,NaN
3,thể,lương,-,61u - 64u,7.64,7.22,0.92
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38


In [1448]:
adverse_reactions.head()

,given_name,surname,adverse_reaction
0,berta,napolitani,injection site discomfort
1,lena,baer,hypoglycemia
2,joseph,day,hypoglycemia
3,flavia,fiorentino,cough
4,manouck,wubbels,throat irritation


# Summary of data 

## Dataset Summary

This dataset focuses on approximately 500 patients, out of which 350 participated in a clinical trial.  
Before the trial, none of these patients had used **Novodra** (a common injectable insulin) or **Auralin** (an oral insulin currently under research) as their insulin source. All participants had elevated HbA1c levels.

To establish baseline measurements, all 350 patients were initially treated with Novodra. After 4 weeks, which was not enough time to affect HbA1c levels significantly, the patients were divided into two equal groups:

- 175 patients switched to **Auralin** for the next 24 weeks  
- 175 patients continued on **Novodra** for 24 weeks

Additionally, the dataset includes records of any **adverse reactions or side effects** reported by the patients during the trial.

### Glossary

- **HbA1c**: Indicates long-term blood sugar level  
- **Novodra**: Injectable insulin  
- **Auralin**: Experimental oral insulin  
- **Adverse reaction**: Negative side effect of a treatment

## Column-wise Description 
#### `patients.csv`, `treatments.csv`, `treatments_cut.csv`, `adverse_reactions.csv`

Before we begin cleaning and analyzing the patient data, it’s important to understand what each column in the dataset represents.

Here is a brief description of all the columns 

# Table  
## Dataset: `patients.csv`

- **`patient_id`**: A unique identifier assigned to every patient in the dataset.
- **`assigned_sex`**: Gender of the patient (either Male or Female).
- **`given_name`**: First name of the patient.
- **`surname`**: Last name (family name) of the patient.
- **`address`**: The full street address of the patient.
- **`city`**: The city in which the patient lives.
- **`state`**: The state of residence of the patient.
- **`zip_code`**: Postal or ZIP code of the patient's area.
- **`country`**: Country where the patient resides.
- **`contact`**: Includes phone number and email ID of the patient.
- **`birthdate`**: Date of birth in MM/DD/YYYY format.
- **`weight`**: Patient's weight in pounds (lbs).
- **`height`**: Patient's height in inches (in).
- **`bmi`**: Body Mass Index, calculated from height and weight.


## `treatments and treatments_cut`

- **`given_name`**: First name of the patient (used to identify individuals).
- **`surname`**: Last name (family name) of the patient.
- **`auralin`**: Number of doses of **Auralin**, an experimental oral insulin, taken by mouth during treatment.
- **`novodra`**: Number of doses of **Novodra**, a standard injectable insulin, taken via injection.
- **`hba1c_start`**: Patient's HbA1c level before starting the treatment.  
  → HbA1c reflects average blood sugar over the past 2–3 months.
- **`hba1c_end`**: Patient's HbA1c level after completing the treatment.  
  → Used to assess effectiveness of the insulin.
- **`hba1c_change`**: Change in HbA1c level during treatment (`hba1c_end - hba1c_start`).  
  → Auralin is considered **noninferior** to Novodra if the upper bound of the 95% Confidence Interval (CI) for the mean difference (Novodra − Auralin) is less than **0.4%**.


## `adverse_reactions.csv`

- **`given_name`**: First name of the patient (used for identification).
- **`surname`**: Last name of the patient.
- **`adverse_reaction`**: Description of any side effect or negative reaction observed during the treatment.


 # 🧹 Data Quality Assessment

This section outlines the issues found in the datasets using both manual inspection and automated code-based analysis.


## 🔍 Manual Assessment

Based on visually inspecting 100–150 rows from each dataset, the following issues were identified:


## 🤖 Programmatic Assessment

In this section, pandas functions are used to automatically detect data quality issues such as missing values, duplicates, incorrect data types, and inconsistent formats.


### 📌 Dataset Preview

We used `.head()` to preview the first few rows of each dataset. This helped in understanding the structure and identifying any obvious data entry issues.


In [1460]:
patients.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [1462]:
treatments.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [1464]:
adverse_reactions.head()

,given_name,surname,adverse_reaction
0,berta,napolitani,injection site discomfort
1,lena,baer,hypoglycemia
2,joseph,day,hypoglycemia
3,flavia,fiorentino,cough
4,manouck,wubbels,throat irritation


In [1465]:
treatments_cut.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,NaN
2,alwin,svensson,36u - 39u,-,7.78,7.34,NaN
3,thể,lương,-,61u - 64u,7.64,7.22,0.92
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38


### 🔢 Shape of the Dataset

Using `.shape` to get the total number of rows and columns.


In [1469]:
patients.shape

(503, 14)

In [1471]:
treatments.shape

(280, 7)

In [1472]:
adverse_reactions.shape

(34, 3)

In [1475]:
treatments_cut.shape

(70, 7)

### 🧾 Column Names

Using `.columns` to display all column names in the dataset.


In [1478]:
patients.columns

Index(['patient_id', 'assigned_sex', 'given_name', 'surname', 'address',
       'city', 'state', 'zip_code', 'country', 'contact', 'birthdate',
       'weight', 'height', 'bmi'],
      dtype='object')

In [1480]:
treatments.columns

Index(['given_name', 'surname', 'auralin', 'novodra', 'hba1c_start',
       'hba1c_end', 'hba1c_change'],
      dtype='object')

In [1482]:
adverse_reactions.columns

Index(['given_name', 'surname', 'adverse_reaction'], dtype='object')

In [1484]:
treatments_cut.columns

Index(['given_name', 'surname', 'auralin', 'novodra', 'hba1c_start',
       'hba1c_end', 'hba1c_change'],
      dtype='object')

### ℹ️ Dataset Info

Using `.info()` to check data types, null counts, and memory usage.


In [1486]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       491 non-null    object 
 5   city          491 non-null    object 
 6   state         491 non-null    object 
 7   zip_code      491 non-null    float64
 8   country       491 non-null    object 
 9   contact       491 non-null    object 
 10  birthdate     503 non-null    object 
 11  weight        503 non-null    float64
 12  height        503 non-null    int64  
 13  bmi           503 non-null    float64
dtypes: float64(3), int64(2), object(9)
memory usage: 55.1+ KB


In [1488]:
treatments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    280 non-null    object 
 1   surname       280 non-null    object 
 2   auralin       280 non-null    object 
 3   novodra       280 non-null    object 
 4   hba1c_start   280 non-null    float64
 5   hba1c_end     280 non-null    float64
 6   hba1c_change  171 non-null    float64
dtypes: float64(3), object(4)
memory usage: 15.4+ KB


In [1491]:
adverse_reactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   given_name        34 non-null     object
 1   surname           34 non-null     object
 2   adverse_reaction  34 non-null     object
dtypes: object(3)
memory usage: 948.0+ bytes


In [1493]:
treatments_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    70 non-null     object 
 1   surname       70 non-null     object 
 2   auralin       70 non-null     object 
 3   novodra       70 non-null     object 
 4   hba1c_start   70 non-null     float64
 5   hba1c_end     70 non-null     float64
 6   hba1c_change  42 non-null     float64
dtypes: float64(3), object(4)
memory usage: 4.0+ KB


### 📊 Statistical Summary

Using `.describe(include='all')` to view summary statistics and detect unusual values.


In [1496]:
patients.describe()

,patient_id,zip_code,weight,height,bmi
count,503.000000,491.000000,503.000000,503.000000,503.000000
mean,252.000000,49084.118126,173.434990,66.634195,27.483897
std,145.347859,30265.807442,33.916741,4.411297,5.276438
min,1.000000,1002.000000,48.800000,27.000000,17.100000
25%,126.500000,21920.500000,149.300000,63.000000,23.300000
50%,252.000000,48057.000000,175.300000,67.000000,27.200000
75%,377.500000,75679.000000,199.500000,70.000000,31.750000
max,503.000000,99701.000000,255.900000,79.000000,37.700000


In [1497]:
treatments.describe()

,hba1c_start,hba1c_end,hba1c_change
count,280.000000,280.000000,171.000000
mean,7.985929,7.589286,0.546023
std,0.568638,0.569672,0.279555
min,7.500000,7.010000,0.200000
25%,7.660000,7.270000,0.340000
50%,7.800000,7.420000,0.380000
75%,7.970000,7.570000,0.920000
max,9.950000,9.580000,0.990000


In [1500]:
treatments_cut.describe(include='all')

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
count,70,70,70,70,70.000000,70.000000,42.000000
unique,70,70,36,31,NaN,NaN,NaN
top,jožka,resanovič,-,-,NaN,NaN,NaN
freq,1,1,32,38,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,7.838000,7.443143,0.518810
std,NaN,NaN,NaN,NaN,0.423007,0.418706,0.270719
min,NaN,NaN,NaN,NaN,7.510000,7.020000,0.280000
25%,NaN,NaN,NaN,NaN,7.640000,7.232500,0.340000
50%,NaN,NaN,NaN,NaN,7.730000,7.345000,0.370000
75%,NaN,NaN,NaN,NaN,7.860000,7.467500,0.907500


In [1502]:
adverse_reactions.describe(include='all')

,given_name,surname,adverse_reaction
count,34,34,34
unique,34,33,6
top,berta,johnson,hypoglycemia
freq,1,2,19


### ❗ Missing Values Check

Using `.isnull().sum()` to count the number of missing values in each column.


In [1505]:
patients.isnull().sum()

patient_id       0
assigned_sex     0
given_name       0
surname          0
address         12
city            12
state           12
zip_code        12
country         12
contact         12
birthdate        0
weight           0
height           0
bmi              0
dtype: int64

In [1507]:
treatments.isnull().sum()

given_name        0
surname           0
auralin           0
novodra           0
hba1c_start       0
hba1c_end         0
hba1c_change    109
dtype: int64

In [1509]:
adverse_reactions.isnull().sum()

given_name          0
surname             0
adverse_reaction    0
dtype: int64

In [1511]:
treatments_cut.isnull().sum()

given_name       0
surname          0
auralin          0
novodra          0
hba1c_start      0
hba1c_end        0
hba1c_change    28
dtype: int64

### 🔁 Duplicate Rows

Using `.duplicated().sum()` to detect any duplicated rows in the dataset.

In [1514]:
patients.duplicated().sum()

0

In [1516]:
treatments.duplicated().sum()

1

In [1518]:
treatments[treatments.duplicated()]

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
136,joseph,day,29u - 36u,-,7.7,7.19,NaN


In [1520]:
adverse_reactions.duplicated().sum()

0

In [1522]:
treatments_cut.duplicated().sum()

0

### 🧠 Category Consistency Check

Using `.value_counts()` or `.unique()` to detect inconsistent labels (like 'M' vs 'Male').


## Patients Table

In [1526]:
print("assigned_sex:\n", patients['assigned_sex'].value_counts())
print("\ncity:\n", patients['city'].value_counts())
print("\nstate:\n", patients['state'].value_counts())
print("\nzip_code:\n", patients['zip_code'].value_counts())
print("\ncountry:\n", patients['country'].value_counts())
print("\ncontact:\n", patients['contact'].value_counts().head(10))


assigned_sex:
 assigned_sex
male      253
female    250
Name: count, dtype: int64

city:
 city
New York         18
San Diego         8
Tulsa             7
Atlanta           6
Houston           6
                 ..
Yazoo City        1
Olathe            1
Winnsboro         1
Chester           1
Overland Park     1
Name: count, Length: 349, dtype: int64

state:
 state
California    36
TX            32
New York      25
CA            24
NY            22
MA            22
PA            18
GA            15
Illinois      14
OH            14
OK            13
MI            13
Florida       13
LA            13
NJ            12
VA            11
MS            10
WI            10
IL            10
IN             9
MN             9
FL             9
AL             9
TN             9
WA             8
NC             8
KY             8
MO             7
ID             6
NV             6
KS             6
SC             5
IA             5
CT             5
ME             4
RI             4
Nebraska       4
ND

## Treatments Table

In [1529]:
print("auralin:\n", treatments['auralin'].value_counts())

auralin:
 auralin
-            143
31u - 38u      3
29u - 37u      3
32u - 42u      2
42u - 49u      2
            ... 
53u - 64u      1
25u - 31u      1
39u - 45u      1
31u - 45u      1
53u - 56u      1
Name: count, Length: 121, dtype: int64


In [1531]:
print("\nnovodra:\n", treatments['novodra'].value_counts())


novodra:
 novodra
-            137
39u - 36u      4
41u - 39u      4
49u - 46u      3
40u - 39u      3
            ... 
41u - 40u      1
35u - 30u      1
37u - 36u      1
38u - 39u      1
49u - 49u      1
Name: count, Length: 113, dtype: int64


## Table adverse_reactions

### Data Redundancy Identified
- The `adverse_reactions` table is **redundant**, as it contains only one additional column (`adverse_reaction`), while `given_name` and `surname` already exist in `treatments`.
- It would be better to **merge this data into the `treatments` table** for cleaner structure.


## Treatments_cut Table

In [1536]:
print("auralin:\n", treatments_cut['auralin'].value_counts())

auralin:
 auralin
-            32
48u - 56u     2
36u - 44u     2
30u - 38u     2
25u - 35u     1
37u - 45u     1
42u - 53u     1
37u - 42u     1
43u - 56u     1
51u - 59u     1
22u - 30u     1
31u - 43u     1
38u - 43u     1
55u - 68u     1
38u - 47u     1
39u - 50u     1
35u - 48u     1
32u - 37u     1
28u - 33u     1
27u - 37u     1
42u - 50u     1
57u - 67u     1
58u - 63u     1
22u - 31u     1
26u - 39u     1
39u - 46u     1
26u - 34u     1
46u - 57u     1
48u - 57u     1
37u - 47u     1
40u - 49u     1
26u - 36u     1
63u - 74u     1
30u - 35u     1
36u - 39u     1
36u - 46u     1
Name: count, dtype: int64


In [1538]:
print("\nnovodra:\n", treatments_cut['novodra'].value_counts())


novodra:
 novodra
-            38
42u - 42u     2
26u - 23u     2
34u - 34u     1
28u - 26u     1
22u - 23u     1
55u - 51u     1
32u - 33u     1
42u - 41u     1
36u - 33u     1
26u - 25u     1
37u - 39u     1
52u - 52u     1
40u - 45u     1
37u - 40u     1
40u - 36u     1
43u - 40u     1
61u - 64u     1
37u - 41u     1
30u - 27u     1
35u - 34u     1
33u - 33u     1
30u - 35u     1
34u - 33u     1
26u - 27u     1
46u - 46u     1
20u - 21u     1
24u - 26u     1
28u - 24u     1
37u - 38u     1
42u - 44u     1
Name: count, dtype: int64


## Data Quality Issues (Manual + Programmatic Combined)

### 🔴 Dirty Data Issues (Labeled by Data Quality Dimensions)
Below are the identified dirty data issues, categorized using standard data quality dimensions:

#### Table -> Patients
### Data Quality Dimension Breakdown

#### **Completeness**
- 12 records are missing in `address`, `city`, and `zip_code` fields.

---

#### **Validity**
- `zip_code` column contains values with less than 5 digits.
- Columns like `birthdate`, `assigned_sex`, `zip_code` have incorrect data types.

---

#### **Accuracy**
- `patient_id` row 9 has a misspelled name: "Dsvid" instead of "David".
- One patient has weight as 48 pounds (unrealistically low).
- One patient has height as 27 inches (unrealistically low).
- Duplicate entries by the name of John Doe.

---

#### **Consistency**
- `state` column has inconsistent formats — mix of full names and abbreviations.

---

#### Table -> Treatments & Treatment_cut
### Data Quality Dimension Breakdown 


#### **Completeness**
- `hba1c_change` column contains missing values.
- 
---

#### **Validity**
- `auralin` and `novodra` columns have entries like "42u - 42u" — unit (`u`) should be removed to retain only numeric dosage.

---

#### **Accuracy**
- `hba1c_change` column has an outlier value `9` instead of expected max around `4`.

---

#### **Consistency**
- Names in `given_name` and `surname` columns are all in lowercase — inconsistent formatting for names.

---

#### Table -> Adverse_reactions 
### Data Quality Dimension Breakdown 
#### **Consistency**
-  Names in `given_name` and `surname` columns are all in lowercase

---
### 🟠 Messy Data
#### Table -> Patients
-  `contact` column contains both phone number and email in one field

---

#### Table -> treatments & Treatment_cut
- `Auralin` and `novadra` col should be split into 2 cols start and end dose

---
#### Table -> Adverse_reactions
- `adverse_reactions` table is unnecessary; can be merged with `treatments`.


## Creating a Copy of the Dataset for Cleaning

Before performing any cleaning operations, we make a copy of the original dataset to preserve the raw data.

This allows safe experimentation and easy comparison with the original version.


#### Create copies for cleaning

In [259]:
patients_clean = patients.copy()
treatments_clean = treatments.copy()
treatments_cut_clean = treatments_cut.copy()
adverse_reactions_clean = adverse_reactions.copy()



### 🔹 Define 1

There are 12 records with missing values in `address`, `city`, or `zip_code` — key fields for identifying patient location. These rows will be cleaned by filling missing values with `"No data"`


**🔹 Code**

In [18]:
# Check missing value
patients_clean[patients_clean['address'].isnull()]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
209,210,female,Lalita,Eldarkhanov,NaN,NaN,NaN,NaN,NaN,NaN,8/14/1950,143.4,62,26.2
219,220,male,Mỹ,Quynh,NaN,NaN,NaN,NaN,NaN,NaN,4/9/1978,237.8,69,35.1
230,231,female,Elisabeth,Knudsen,NaN,NaN,NaN,NaN,NaN,NaN,9/23/1976,165.9,63,29.4
234,235,female,Martina,Tománková,NaN,NaN,NaN,NaN,NaN,NaN,4/7/1936,199.5,65,33.2
242,243,male,John,O'Brian,NaN,NaN,NaN,NaN,NaN,NaN,2/25/1957,205.3,74,26.4
249,250,male,Benjamin,Mehler,NaN,NaN,NaN,NaN,NaN,NaN,10/30/1951,146.5,69,21.6
257,258,male,Jin,Kung,NaN,NaN,NaN,NaN,NaN,NaN,5/17/1995,231.7,69,34.2
264,265,female,Wafiyyah,Asfour,NaN,NaN,NaN,NaN,NaN,NaN,11/3/1989,158.6,63,28.1
269,270,female,Flavia,Fiorentino,NaN,NaN,NaN,NaN,NaN,NaN,10/9/1937,175.2,61,33.1
278,279,female,Generosa,Cabán,NaN,NaN,NaN,NaN,NaN,NaN,12/16/1962,124.3,69,18.4


In [ ]:
patients_clean.fillna('No data',inplace=True)

 **🔹 Verifying the update** 

In [26]:
patients_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       503 non-null    object 
 5   city          503 non-null    object 
 6   state         503 non-null    object 
 7   zip_code      503 non-null    object 
 8   country       503 non-null    object 
 9   contact       503 non-null    object 
 10  birthdate     503 non-null    object 
 11  weight        503 non-null    float64
 12  height        503 non-null    int64  
 13  bmi           503 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 55.1+ KB


### 🔹 Define 2
Missing hba1c_change values were calculated using hba1c_start - hba1c_end . 

**🔹 Code**

In [30]:
treatments.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [32]:
# Create new column for hba1c change (start - end)
treatments_clean['hba1c_change'] = treatments_clean['hba1c_start'] - treatments_clean['hba1c_end']
treatments_cut_clean['hba1c_change'] = treatments_cut_clean['hba1c_start'] - treatments_cut_clean['hba1c_end']

 **🔹 Verifying the update** 

In [35]:
treatments_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    280 non-null    object 
 1   surname       280 non-null    object 
 2   auralin       280 non-null    object 
 3   novodra       280 non-null    object 
 4   hba1c_start   280 non-null    float64
 5   hba1c_end     280 non-null    float64
 6   hba1c_change  280 non-null    float64
dtypes: float64(3), object(4)
memory usage: 15.4+ KB


In [37]:
treatments_cut_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    70 non-null     object 
 1   surname       70 non-null     object 
 2   auralin       70 non-null     object 
 3   novodra       70 non-null     object 
 4   hba1c_start   70 non-null     float64
 5   hba1c_end     70 non-null     float64
 6   hba1c_change  70 non-null     float64
dtypes: float64(3), object(4)
memory usage: 4.0+ KB


### 🔹 Define 3
contact column contains both email and phone number. We need to split them into two separate columns

**🔹 code** 

In [220]:

def find_contact_details(text: str) -> tuple:
    # If input is NaN, return NaN
    if pd.isna(text):
        return np.nan

    # Compile the phone number pattern (with or without country code)
    phone_number_pattern = re.compile(r"(\+[\d]{1,3}\s?)?(\(?\d{3}\)?[-\s]?\d{3}[-\s]?\d{4})")

    # Find all matches (will return list of tuples)
    matches = re.findall(phone_number_pattern, text)

    if len(matches) == 0:
        phone_number = np.nan
    else:
        # Combine both groups (country code + number) if present
        match = matches[0]
        phone_number = ''.join(match).strip()

    # Remove the phone number from original text to isolate possible email
    cleaned_text = re.sub(phone_number_pattern, "", text).strip()

    # Return tuple: (phone, possible email)
    return phone_number, cleaned_text

In [222]:
# Split contact into phone and email columns
patients_clean[['phone', 'email']] = patients_clean['contact'].apply(
    lambda x: pd.Series(find_contact_details(x))
)

In [224]:
patients_clean[['contact', 'phone', 'email']].head(10)

,contact,phone,email
0,951-719-9170ZoeWellish@superrito.com,951-719-9170,ZoeWellish@superrito.com
1,PamelaSHill@cuvox.de+1 (217) 569-3204,+1 (217) 569-3204,PamelaSHill@cuvox.de
2,402-363-6804JaeMDebord@gustr.com,402-363-6804,JaeMDebord@gustr.com
3,PhanBaLiem@jourrapide.com+1 (732) 636-8246,+1 (732) 636-8246,PhanBaLiem@jourrapide.com
4,334-515-7487TimNeudorf@cuvox.de,334-515-7487,TimNeudorf@cuvox.de
5,386-334-5237RafaelCardosoCosta@gustr.com,386-334-5237,RafaelCardosoCosta@gustr.com
6,775-533-5933MaryBAdams@einrot.com,775-533-5933,MaryBAdams@einrot.com
7,XiuxiuChang@einrot.com1 408 778 3236,408 778 3236,XiuxiuChang@einrot.com1
8,816-265-9578DavidGustafsson@armyspy.com,816-265-9578,DavidGustafsson@armyspy.com
9,SophieCabreraIbarra@teleworm.us1 718 795 9124,718 795 9124,SophieCabreraIbarra@teleworm.us1


In [226]:
# drop contact column
patients_clean.drop(columns=['contact'], inplace=True)

**🔹 Verifying the update** 

In [229]:
patients_clean

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,zip_len,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390,United States,1976-07-10,121.7,66,19.6,5,951-719-9170,ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812,United States,1967-04-03,118.8,66,19.2,5,+1 (217) 569-3204,PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467,United States,1980-02-19,177.8,71,24.8,5,402-363-6804,JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,New Jersey,07095,United States,1951-07-26,220.9,70,31.7,4,+1 (732) 636-8246,PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,Alabama,36303,United States,1928-02-18,192.3,67,26.1,5,334-515-7487,TimNeudorf@cuvox.de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,Maine,03852,United States,1959-04-10,181.1,72,24.6,4,207-477-0579,MustafaLindstrom@jourrapide.com
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,Arizona,86341,United States,1948-03-26,239.6,70,34.4,5,928-284-4492,RumanBisliev@gustr.com
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,Missouri,64110,United States,1971-01-13,171.2,67,26.8,5,816-223-6007,JinkedeKeizer@teleworm.us
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,Washington,98109,United States,1952-02-13,176.9,67,27.7,5,360 443 2060,ChidaluOnyekaozulu@jourrapide.com1


### 🔹 Define 4
### 🔹 Merge `treatments` and `treatment_cut` Tables

We are merging the `treatments` table with the `treatment_cut` table to bring all relevant information together.

**🔹 Code**

In [267]:
# concat the columns
treatments_clean = pd.concat([treatments_clean,treatments_cut_clean])

In [269]:
# Melt safely
treatments_clean = treatments_clean.melt(id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'],var_name='type',value_name='dosage_info')

In [271]:
print(treatments_clean.columns)


Index(['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change',
       'type', 'dosage_info'],
      dtype='object')


In [272]:
treatments_clean = treatments_clean[treatments_clean['dosage_info'] != '-']

In [275]:
treatments_clean['dosage_start'] = treatments_clean['dosage_info'].str.split('-').str.get(0)
treatments_clean['dosage_end'] = treatments_clean['dosage_info'].str.split('-').str.get(1)

In [277]:
treatments_clean.drop(columns='dosage_info',inplace=True)

 **🔹 Verifying the update** 
 
 Let's check the first few rows and shape of the merged dataframe to confirm the merge worked correctly.

In [280]:
treatments_clean

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_start,dosage_end
0,veronika,jindrová,7.63,7.20,NaN,auralin,41u,48u
3,skye,gormanston,7.97,7.62,0.35,auralin,33u,36u
6,sophia,haugen,7.65,7.27,0.38,auralin,37u,42u
7,eddie,archer,7.89,7.55,0.34,auralin,31u,38u
9,asia,woźniak,7.76,7.37,NaN,auralin,30u,36u
...,...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.95,novodra,55u,51u
690,maret,sultygov,7.67,7.30,NaN,novodra,26u,23u
694,lixue,hsueh,9.21,8.80,0.91,novodra,22u,23u
696,jakob,jakobsen,7.96,7.51,0.95,novodra,28u,26u


### 🔹 Define 5
The adverse_reactions table is not needed as a separate table; it can be merged with the treatments table to simplify analysis and reduce redundancy.

**🔹 Code**

In [284]:
# Merge adverse reactions
treatments_clean.merge = treatments_clean.merge(adverse_reactions_clean, how = 'left', on=['given_name','surname'])

**🔹 Verifying the update** 

In [287]:
treatments_clean.merge

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_start,dosage_end,adverse_reaction
0,veronika,jindrová,7.63,7.20,NaN,auralin,41u,48u,NaN
1,skye,gormanston,7.97,7.62,0.35,auralin,33u,36u,NaN
2,sophia,haugen,7.65,7.27,0.38,auralin,37u,42u,NaN
3,eddie,archer,7.89,7.55,0.34,auralin,31u,38u,NaN
4,asia,woźniak,7.76,7.37,NaN,auralin,30u,36u,NaN
...,...,...,...,...,...,...,...,...,...
345,christopher,woodward,7.51,7.06,0.95,novodra,55u,51u,nausea
346,maret,sultygov,7.67,7.30,NaN,novodra,26u,23u,NaN
347,lixue,hsueh,9.21,8.80,0.91,novodra,22u,23u,injection site discomfort
348,jakob,jakobsen,7.96,7.51,0.95,novodra,28u,26u,hypoglycemia


### 🔹 Define 6
The auralin and novodra columns contain dosage values like "42u - 42u". To standardize the data for numerical analysis, the unit "u" should be removed so that only numeric dosage values remain.



**🔹 Code**

In [291]:
# Remove 'u' from dosage columns
treatments_clean['dosage_start'] = treatments_clean['dosage_start'].str.replace('u','')
treatments_clean['dosage_end'] = treatments_clean['dosage_end'].str.replace('u','')

**🔹 Verifying the update** 

In [294]:
treatments_clean

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_start,dosage_end
0,veronika,jindrová,7.63,7.20,NaN,auralin,41,48
3,skye,gormanston,7.97,7.62,0.35,auralin,33,36
6,sophia,haugen,7.65,7.27,0.38,auralin,37,42
7,eddie,archer,7.89,7.55,0.34,auralin,31,38
9,asia,woźniak,7.76,7.37,NaN,auralin,30,36
...,...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.95,novodra,55,51
690,maret,sultygov,7.67,7.30,NaN,novodra,26,23
694,lixue,hsueh,9.21,8.80,0.91,novodra,22,23
696,jakob,jakobsen,7.96,7.51,0.95,novodra,28,26


### 🔹 Define 7
The dosage_start, and dosage_end columns are currently in object data type. Since dosage_start and dosage_end contain numeric values, they should be converted to int type for accurate analysis and calculations.

In [84]:
treatments_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 350 entries, 0 to 698
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    350 non-null    object 
 1   surname       350 non-null    object 
 2   hba1c_start   350 non-null    float64
 3   hba1c_end     350 non-null    float64
 4   hba1c_change  350 non-null    float64
 5   type          350 non-null    object 
 6   dosage_start  350 non-null    object 
 7   dosage_end    350 non-null    object 
dtypes: float64(3), object(5)
memory usage: 24.6+ KB


**🔹 Code**

In [87]:
# fix dtype
treatments_clean['dosage_start'] = treatments_clean['dosage_start'].astype('int')
treatments_clean['dosage_end'] = treatments_clean['dosage_end'].astype('int')

**🔹 Verifying the update** 

In [90]:
treatments_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 350 entries, 0 to 698
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    350 non-null    object 
 1   surname       350 non-null    object 
 2   hba1c_start   350 non-null    float64
 3   hba1c_end     350 non-null    float64
 4   hba1c_change  350 non-null    float64
 5   type          350 non-null    object 
 6   dosage_start  350 non-null    int32  
 7   dosage_end    350 non-null    int32  
dtypes: float64(3), int32(2), object(3)
memory usage: 21.9+ KB


### 🔹 Define 8
Merging Treatment and Adverse Reaction Data

**🔹 Code**

In [99]:
# Merging Treatment and Adverse Reaction tables
treatments_clean = treatments_clean.merge(adverse_reactions_clean, how = 'left', on=['given_name','surname'])

**🔹 Verifying the update**

In [102]:
treatments_clean

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change,adverse_reaction
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97,hypoglycemia
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35,NaN
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32,NaN
...,...,...,...,...,...,...,...,...
275,albina,zetticci,45u - 51u,-,7.93,7.73,0.20,NaN
276,john,teichelmann,-,49u - 49u,7.90,7.58,NaN,NaN
277,mathea,lillebø,23u - 36u,-,9.04,8.67,0.37,NaN
278,vallie,prince,31u - 38u,-,7.64,7.28,0.36,NaN


### 🔹 Define 9
The zip_code column contains inconsistent values — some entries have fewer than 5 digits.

**🔹 Code**

In [106]:
# fix dtype
patients_clean['zip_code'] = patients_clean['zip_code'].astype('Int64')
patients_clean['zip_len'] = patients_clean['zip_code'].astype(str).str.len()

print(patients_clean['zip_len'].value_counts())

zip_len
5    442
4     61
Name: count, dtype: int64


In [108]:
# Format zip_code with leading zeros
patients_clean['zip_code'] = patients_clean['zip_code'].astype('Int64').astype(str).str.zfill(5)

**🔹 Verifying the update** 

In [111]:
patients_clean['zip_code'].astype(str).str.len().value_counts()

zip_code
5    503
Name: count, dtype: int64

### 🔹 Define 10
Several columns in the dataset have incorrect data types. Below is the explanation and the corrections applied:

**🔹 Code**

In [115]:
# fix dtype
patients_clean['assigned_sex'] = patients_clean['assigned_sex'].astype('category')
patients_clean['birthdate'] = patients_clean['birthdate'].astype('datetime64[ns]')

**🔹 Verifying the update** 

In [118]:
patients_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   patient_id    503 non-null    int64         
 1   assigned_sex  503 non-null    category      
 2   given_name    503 non-null    object        
 3   surname       503 non-null    object        
 4   address       491 non-null    object        
 5   city          491 non-null    object        
 6   state         491 non-null    object        
 7   zip_code      503 non-null    object        
 8   country       491 non-null    object        
 9   contact       491 non-null    object        
 10  birthdate     503 non-null    datetime64[ns]
 11  weight        503 non-null    float64       
 12  height        503 non-null    int64         
 13  bmi           503 non-null    float64       
 14  zip_len       503 non-null    int64         
dtypes: category(1), datetime64[ns](1), float

### 🔹 Define 11
A misspelled value refers to a typographical or spelling error in a data entry.
For example, "Dsvid" instead of "David" in a name column is a data quality issue. Such errors can impact grouping, filtering, or joining datasets.

**🔹 Code**

In [122]:
patients_clean[patients_clean['patient_id'] == 9].head(1)

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,zip_len
8,9,male,Dsvid,Gustafsson,1790 Nutter Street,Kansas City,MO,64105,United States,816-265-9578DavidGustafsson@armyspy.com,1937-03-06,163.9,66,26.5,5


In [123]:
# fix name for patient_id 8
patients_clean.loc[patients_clean['patient_id'] == 8, 'given_name'] = 'David'

**🔹 Verifying the update** 

In [127]:
patients_clean[patients_clean['patient_id'] == 8].head(1)


,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,zip_len
7,8,female,David,Chang,2687 Black Oak Hollow Road,Morgan Hill,CA,95037,United States,XiuxiuChang@einrot.com1 408 778 3236,1958-08-13,158.0,60,30.9,5


### 🔹 Define 12
One patient has a **weight of 48 pounds**, which is extremely low for an adult.

One patient has a **height of 27 inches**, which is also unrealistically low.

**🔹 Code**

To maintain data integrity and avoid loss of information, we replaced these unrealistic values with the **median** of their respective columns:


In [131]:
# Calculate median of weight and height
median_weight = patients_clean['weight'].median()
median_height = patients_clean['height'].median()

In [133]:
print(median_weight)
print(median_height)

175.3
67.0


In [135]:
patients_clean.loc[patients_clean['weight'] == 48.8, 'weight'] = median_weight
patients_clean.loc[patients_clean['height'] == 27.0, 'height'] = median_height 

**🔹 Verifying the update** 

In [138]:
patients_clean.describe()

,patient_id,birthdate,weight,height,bmi,zip_len
count,503.000000,503,503.000000,503.000000,503.000000,503.000000
mean,252.000000,1959-02-03 15:33:16.819085504,173.686481,66.713718,27.483897,4.878728
min,1.000000,1921-11-06 00:00:00,102.100000,59.000000,17.100000,4.000000
25%,126.500000,1938-04-13 00:00:00,149.500000,63.000000,23.300000,5.000000
50%,252.000000,1959-04-10 00:00:00,175.300000,67.000000,27.200000,5.000000
75%,377.500000,1978-04-16 00:00:00,199.500000,70.000000,31.750000,5.000000
max,503.000000,1999-07-03 00:00:00,255.900000,79.000000,37.700000,5.000000
std,145.347859,NaN,33.456612,4.040328,5.276438,0.326769


In [140]:
patients_clean.loc[patients_clean['weight'] == 48.8]
patients_clean.loc[patients_clean['height'] == 27.0]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi,zip_len


### 🔹 Define 13

Duplicate entries by the name of John Doe Fix it 

**🔹 Code**

# Locate duplicate entry (John Doe)
patients_clean[(patients_clean['given_name'] == 'John') & (patients_clean['surname'] == 'Doe')]

In [145]:
patients_clean.drop_duplicates(subset=['given_name', 'surname'], keep='first', inplace=True)


**🔹 Verifying the update** 

In [148]:
patients_clean[(patients_clean['given_name'] == 'John') & (patients_clean['surname'] == 'Doe')].shape[0]


1

### 🔹 Define 14
hba1c_change column has an outlier value 9 instead of expected max around 4.

**🔹 Code**

In [152]:
# Sort hba1c_change
treatments_clean.sort_values('hba1c_change',na_position='first')

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change,adverse_reaction
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN,NaN
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN,NaN
8,saber,ménard,-,54u - 54u,8.08,7.70,NaN,NaN
9,asia,woźniak,30u - 36u,-,7.76,7.37,NaN,NaN
10,joseph,day,29u - 36u,-,7.70,7.19,NaN,hypoglycemia
...,...,...,...,...,...,...,...,...
49,jackson,addison,-,42u - 42u,7.99,7.51,0.98,NaN
17,gina,cain,-,36u - 36u,7.88,7.40,0.98,NaN
32,laura,ehrlichmann,-,43u - 40u,7.95,7.46,0.99,NaN
245,wu,sung,-,47u - 48u,7.61,7.12,0.99,NaN


In [154]:
treatments_clean['hba1c_change'] = (treatments_clean['hba1c_start'] - treatments_clean['hba1c_end']).round(2)


**🔹 Verifying the update** 

In [157]:
treatments_clean.sort_values('hba1c_change',na_position='first')

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change,adverse_reaction
275,albina,zetticci,45u - 51u,-,7.93,7.73,0.20,NaN
70,tosh,jensen,-,51u - 48u,7.93,7.69,0.24,NaN
237,manouck,wubbels,55u - 62u,-,7.66,7.40,0.26,throat irritation
156,chidalu,onyekaozulu,-,42u - 41u,7.54,7.27,0.27,NaN
144,nicoline,østergaard,24u - 32u,-,7.84,7.57,0.27,NaN
...,...,...,...,...,...,...,...,...
273,kate,wilkinson,36u - 39u,-,7.72,7.20,0.52,NaN
187,león,reynoso,-,38u - 40u,7.59,7.06,0.53,NaN
51,kristoffer,martinsen,29u - 37u,-,9.18,8.64,0.54,NaN
178,kerman,dandonneau,41u - 50u,-,7.82,7.28,0.54,NaN


### 🔹 Define 15
state column has inconsistent formats — mix of full names and abbreviations.

**🔹 Code**

In [161]:
# City/state count
print("\ncity:\n", patients_clean['city'].value_counts())
print("\nstate:\n", patients_clean['state'].value_counts())


city:
 city
New York         13
San Diego         8
Tulsa             7
Atlanta           6
Houston           6
                 ..
Yazoo City        1
Olathe            1
Winnsboro         1
Chester           1
Overland Park     1
Name: count, Length: 349, dtype: int64

state:
 state
California    36
TX            32
New York      25
CA            24
MA            22
PA            18
NY            17
GA            15
Illinois      14
OH            14
OK            13
MI            13
Florida       13
LA            13
NJ            12
VA            11
MS            10
WI            10
IL            10
IN             9
MN             9
FL             9
AL             9
TN             9
WA             8
NC             8
KY             8
MO             7
ID             6
NV             6
KS             6
SC             5
IA             5
CT             5
ME             4
RI             4
Nebraska       4
ND             4
CO             4
AZ             4
AR             4
MD             3

In [163]:
# Create a Mapping Dictionary
state_abbrev = {
    'CA': 'California',
    'NY': 'New York',
    'TX': 'Texas',
    'FL': 'Florida',
    'IL': 'Illinois',
    'PA': 'Pennsylvania',
    'OH': 'Ohio',
    'GA': 'Georgia',
    'NC': 'North Carolina',
    'MI': 'Michigan',
    'NJ': 'New Jersey',
    'VA': 'Virginia',
    'WA': 'Washington',
    'AZ': 'Arizona',
    'MA': 'Massachusetts',
    'TN': 'Tennessee',
    'IN': 'Indiana',
    'MO': 'Missouri',
    'MD': 'Maryland',
    'WI': 'Wisconsin',
    'CO': 'Colorado',
    'MN': 'Minnesota',
    'SC': 'South Carolina',
    'AL': 'Alabama',
    'LA': 'Louisiana',
    'KY': 'Kentucky',
    'OR': 'Oregon',
    'OK': 'Oklahoma',
    'CT': 'Connecticut',
    'IA': 'Iowa',
    'MS': 'Mississippi',
    'AR': 'Arkansas',
    'KS': 'Kansas',
    'UT': 'Utah',
    'NV': 'Nevada',
    'NM': 'New Mexico',
    'NE': 'Nebraska',
    'WV': 'West Virginia',
    'ID': 'Idaho',
    'HI': 'Hawaii',
    'NH': 'New Hampshire',
    'ME': 'Maine',
    'RI': 'Rhode Island',
    'MT': 'Montana',
    'DE': 'Delaware',
    'SD': 'South Dakota',
    'ND': 'North Dakota',
    'AK': 'Alaska',
    'VT': 'Vermont',
    'WY': 'Wyoming',
    'DC': 'District of Columbia'
}


In [165]:
# Replace abbreviation with full name
patients_clean['state'] = patients_clean['state'].apply(lambda x: state_abbrev.get(x, x))

**🔹 Verifying the update** 

In [168]:
print("\ncity:\n", patients_clean['city'].value_counts())
print("\nstate:\n", patients_clean['state'].value_counts())


city:
 city
New York         13
San Diego         8
Tulsa             7
Atlanta           6
Houston           6
                 ..
Yazoo City        1
Olathe            1
Winnsboro         1
Chester           1
Overland Park     1
Name: count, Length: 349, dtype: int64

state:
 state
California              60
New York                42
Texas                   32
Illinois                24
Florida                 22
Massachusetts           22
Pennsylvania            18
Georgia                 15
Ohio                    14
Michigan                13
Oklahoma                13
Louisiana               13
New Jersey              12
Virginia                11
Wisconsin               10
Mississippi             10
Alabama                  9
Tennessee                9
Indiana                  9
Minnesota                9
North Carolina           8
Kentucky                 8
Washington               8
Missouri                 7
Nebraska                 6
Kansas                   6
Idaho      

### 🔹 Define 16
Names in given_name and surname columns are all in lowercase

**🔹 code** 

In [172]:
print(treatments_cut_clean[['given_name', 'surname']].head(10))

  given_name     surname
0      jožka   resanovič
1  inunnguaq    heilmann
2      alwin    svensson
3        thể       lương
4     amanda     ribeiro
5       meya     sjöberg
6       sara       miles
7    frydryk     adamski
8       jane     citizen
9     angela  lavrentyev


In [174]:
treatments_cut_clean['given_name'] = patients_clean['given_name'].str.title()
treatments_cut_clean['surname'] = patients_clean['surname'].str.title()

**🔹 Verifying the update** 

In [177]:
treatments_cut_clean

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,Zoe,Wellish,22u - 30u,-,7.56,7.22,0.34
1,Pamela,Hill,57u - 67u,-,7.85,7.45,NaN
2,Jae,Debord,36u - 39u,-,7.78,7.34,NaN
3,Liêm,Phan,-,61u - 64u,7.64,7.22,0.92
4,Tim,Neudorf,36u - 44u,-,7.85,7.47,0.38
...,...,...,...,...,...,...,...
65,Nicolas,Ferreira,32u - 37u,-,7.75,7.41,0.34
66,Konstantyn,Kowalczyk,-,28u - 26u,7.96,7.51,0.95
67,Nebechi,Ekechukwu,48u - 56u,-,7.74,7.44,0.30
68,Haruna,Azuma,-,42u - 44u,7.68,7.21,NaN


### 🔹 Define 17
####  Melting `treatments_cut_clean`

Useful for cleaner data analysis and visualization.


**🔹 code** 

In [233]:
# Melt safely
treatments_cut_clean = treatments_cut_clean.melt(
    id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'],
    var_name='type',
    value_name='dosage_info'
)

In [235]:
print(treatments_cut_clean.columns)


Index(['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change',
       'type', 'dosage_info'],
      dtype='object')


In [237]:
treatments_cut_clean = treatments_cut_clean[treatments_cut_clean['dosage_info'] != '-']

In [239]:
treatments_cut_clean['dosage_start'] = treatments_cut_clean['dosage_info'].str.split('-').str.get(0)

In [241]:

treatments_cut_clean['dosage_end'] = treatments_cut_clean['dosage_info'].str.split('-').str.get(1)

In [243]:
treatments_cut_clean.drop(columns='dosage_info',inplace=True)

**🔹 Verifying the update** 

In [246]:
treatments_cut_clean

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_start,dosage_end
0,Zoe,Wellish,7.56,7.22,0.34,type,auralin,NaN
1,Pamela,Hill,7.85,7.45,NaN,type,auralin,NaN
2,Jae,Debord,7.78,7.34,NaN,type,auralin,NaN
3,Tim,Neudorf,7.85,7.47,0.38,type,auralin,NaN
4,Mary,Adams,7.53,7.12,NaN,type,auralin,NaN
...,...,...,...,...,...,...,...,...
205,Yasmin,Araujo,7.51,7.06,0.95,dosage_end,NaN,NaN
206,Barbora,Vesecká,7.67,7.30,NaN,dosage_end,NaN,NaN
207,Jaakko,Eskelinen,9.21,8.80,0.91,dosage_end,NaN,NaN
208,Konstantyn,Kowalczyk,7.96,7.51,0.95,dosage_end,NaN,NaN


### 🔹 Define 18
#### treatments_cut_clean Table
The auralin and novodra columns contain dosage values like "42u - 42u". To standardize the data for numerical analysis, the unit "u" should be removed so that only numeric dosage values remain.



**🔹 code** 

In [ ]:
# Remove 'u' from dosage values
treatments_cut_clean['dosage_start'] = treatments_cut_clean['dosage_start'].str.replace('u','')
treatments_cut_clean['dosage_end'] = treatments_cut_clean['dosage_end'].str.replace('u','')

**🔹 Verifying the update** 

In [ ]:
treatments_cut_clean

### 🔹 Define 19

**🔹 code** 
#### Fixing Incorrect Data Types

The dosage_start and dosage_end columns had incorrect data types (`object`).  

In [205]:
# fix dtype
treatments_cut_clean['dosage_start'] = treatments_cut_clean['dosage_start'].astype('float64')
treatments_cut_clean['dosage_end'] = treatments_cut_clean['dosage_end'].astype('float64')

**🔹 Verifying the update** 

In [208]:
treatments_cut_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 0 to 138
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    70 non-null     object 
 1   surname       70 non-null     object 
 2   hba1c_start   70 non-null     float64
 3   hba1c_end     70 non-null     float64
 4   hba1c_change  42 non-null     float64
 5   type          70 non-null     object 
 6   dosage_start  70 non-null     float64
 7   dosage_end    70 non-null     float64
dtypes: float64(5), object(3)
memory usage: 4.9+ KB


# Display first few rows of each cleaned table

In [231]:
patients_clean

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,zip_len,phone,email
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390,United States,1976-07-10,121.7,66,19.6,5,951-719-9170,ZoeWellish@superrito.com
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812,United States,1967-04-03,118.8,66,19.2,5,+1 (217) 569-3204,PamelaSHill@cuvox.de
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467,United States,1980-02-19,177.8,71,24.8,5,402-363-6804,JaeMDebord@gustr.com
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,New Jersey,07095,United States,1951-07-26,220.9,70,31.7,4,+1 (732) 636-8246,PhanBaLiem@jourrapide.com
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,Alabama,36303,United States,1928-02-18,192.3,67,26.1,5,334-515-7487,TimNeudorf@cuvox.de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,Maine,03852,United States,1959-04-10,181.1,72,24.6,4,207-477-0579,MustafaLindstrom@jourrapide.com
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,Arizona,86341,United States,1948-03-26,239.6,70,34.4,5,928-284-4492,RumanBisliev@gustr.com
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,Missouri,64110,United States,1971-01-13,171.2,67,26.8,5,816-223-6007,JinkedeKeizer@teleworm.us
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,Washington,98109,United States,1952-02-13,176.9,67,27.7,5,360 443 2060,ChidaluOnyekaozulu@jourrapide.com1


In [296]:
treatments_clean

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_start,dosage_end
0,veronika,jindrová,7.63,7.20,NaN,auralin,41,48
3,skye,gormanston,7.97,7.62,0.35,auralin,33,36
6,sophia,haugen,7.65,7.27,0.38,auralin,37,42
7,eddie,archer,7.89,7.55,0.34,auralin,31,38
9,asia,woźniak,7.76,7.37,NaN,auralin,30,36
...,...,...,...,...,...,...,...,...
688,christopher,woodward,7.51,7.06,0.95,novodra,55,51
690,maret,sultygov,7.67,7.30,NaN,novodra,26,23
694,lixue,hsueh,9.21,8.80,0.91,novodra,22,23
696,jakob,jakobsen,7.96,7.51,0.95,novodra,28,26


In [218]:
treatments_cut_clean

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,type,dosage_start,dosage_end
0,Zoe,Wellish,7.56,7.22,0.34,auralin,22.0,30.0
1,Pamela,Hill,7.85,7.45,NaN,auralin,57.0,67.0
2,Jae,Debord,7.78,7.34,NaN,auralin,36.0,39.0
4,Tim,Neudorf,7.85,7.47,0.38,auralin,36.0,44.0
6,Mary,Adams,7.53,7.12,NaN,auralin,30.0,35.0
...,...,...,...,...,...,...,...,...
128,Yasmin,Araujo,7.51,7.06,0.95,novodra,55.0,51.0
130,Barbora,Vesecká,7.67,7.30,NaN,novodra,26.0,23.0
134,Jaakko,Eskelinen,9.21,8.80,0.91,novodra,22.0,23.0
136,Konstantyn,Kowalczyk,7.96,7.51,0.95,novodra,28.0,26.0


## 🛠️ Key Cleaning Actions Performed
As the raw datasets were messy and inconsistent, the following key data cleaning steps were performed:


### Data Cleaning Overview

- ✅ Removed duplicates and invalid records.
- ✅ Filled or handled missing values using fillna() or by filtering out.
- ✅ Standardized formats for zip codes, names, and states.
- ✅ Converted column data types where necessary.
- ✅ Cleaned and split dosage columns (auralin, novodra) into *_start and *_end.
- ✅ Dropped redundant or unusable columns like contact.
- ✅ Merged adverse_reactions_clean with treatments_clean.

In [299]:
patients_clean.to_csv("patients_cleaned.csv", index=False)
treatments_clean.to_csv("treatments_cleaned.csv", index=False)
treatments_cut_clean.to_csv("treatments_cut_cleaned.csv", index=False)